In [1]:
%%writefile Scheduler.py

import datetime as dt
import time as tm

# class Scheduler의 객체들은 년/월/일/시간/요일/스케줄내용/중요한일정표시/D-day 의 속성들을 갖게 된다.
class Scheduler:
    # Scheduler 클래스로 생성한 객체들을 관리하기 위한 dictionary를 만들었다.
    schedules = {}
    
    def __init__(self, year, month, day, hour, minute, dayname, schedule):
        self._year, self._month, self._day, self._dayname = year, month, day, dayname
        self._schedule = schedule
        self._hour = hour
        self._minute = minute
        Scheduler.schedules[self] = self._schedule

        self._mark = None
        self._dday = None

        print("=> {}년 {}월 {}일 {} {}시 {}분 \"{}\" 일정이 등록되었습니다."
              .format(self._year, self._month, self._day, self._dayname, self._hour, self._minute, self._schedule))
        print("----------------------------------------------------------\n")

    # 객체의 속성 중 하나인 _dayname을 얻기 위한 과정이 길어서,
    # gen_date 메쏘드를 이용하여 메쏘드를 콜하는 방식으로 객체를 생성하려 한다. 이 때, staticmethod를 활용
    @staticmethod
    def gen_schedule():
        # schedule 정보를 입력 받는다.
        while True:
            try:
                year = int(input("__년 : "))
                if year >= 0:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue
        while True:
            try:
                month = int(input("__월 : "))
                if 1 <= month <= 12:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue
        while True:
            dic_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
            try:
                day = int(input("__일 : "))
                if 1 <= day <= dic_month[month]:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue
        while True:
            try:
                hour = int(input("__시(0~23) : "))
                if 1 <= hour <= 23:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue
        while True:
            try:
                minute = int(input("__분 : "))
                if 0 <= minute <= 59:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue

        schedule = input("일정 내용: ")

        # dayname(요일)을 생성하기 위한 함수들 -> 윤년구하기 OPP v1. 활용
        def is_leap(year):
            check = False
            if year % 4 == 0:
                if year % 100 == 0:
                    if year % 400 == 0:
                        check = True
                    else:
                        pass
                else:
                    check = True

            return check

        # year의 직전년도까지 있었던 윤년의 개수를 구한다.
        def countleap_till_lastyear(year):
            count = 0
            for i in range(0, year, 4):
                if i == 0:
                    pass
                else:
                    if i % 100 == 0:
                        if i % 400 == 0:
                            count += 1
                        else:
                            pass
                    else:
                        count += 1

            return count

        # year년도의 1월1일부터 입력한 날짜까지의 day 개수를 구한다.
        def numofdays_thisyear(year, month, day):
            dic_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
            count = 0
            for i in range(1, month):
                count += dic_month[i]
            count = count + day

            if (is_leap(year) is True) and (month >= 3):
                count += 1

            return count

        # 입력한 year, month, day에 맞는 dayname을 구한다.
        def get_day_name(year, month, day):
            days = ['일요일', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일']
            lastyear = year - 1

            days_till_today = lastyear * 365 + countleap_till_lastyear(year) + numofdays_thisyear(year, month, day)
            dayname = days[days_till_today % 7]

            return dayname

        dayname = get_day_name(year, month, day)

        return Scheduler(year, month, day, hour, minute, dayname, schedule)

    # 중요한 일정에는 (*중요*)마크를 해준다
    def gen_mark(self):
        self._mark = "(*중요*)"
        Scheduler.schedules[self] = self._schedule
        print("일정에 (*중요*) 표시를 하였습니다.")
        
    # 등록한 일정 날짜와 현재 날짜로의 timedelta를 계산해 d-day를 구한다.
    def gen_dday(self):
        today = dt.datetime.today()
        delta = dt.datetime(self._year, self._month, self._day) - dt.datetime(today.year, today.month,
                                                                              today.day)
        self._dday = "d - " + str(delta.days)
        Scheduler.schedules[self] = self._schedule
        if self._mark is not None:
            print(self._mark, end='')
        print("\"%s\" 일정까지 %s, 설정 완료" % (self._schedule, self._dday))

        
    # 외부에서 schedule을 attribute인 것처럼 사용해 접근할 수 있도록 property 활용
    def get_schedule(self):
        return self._schedule

    def set_schedule(self, schedule):
        self._schedule = schedule
        Scheduler.schedules[self] = self._schedule

        print("=> {}년 {}월 {}일 {} {}시 {}분 \"{}\" 일정으로 변경되었습니다."
              .format(self._year, self._month, self._day, self._dayname, self._hour, self._minute, self._schedule))
        print("----------------------------------------------------------\n")
        

    schedule = property(get_schedule, set_schedule)

    # 아래의 변수들은 외부에서 일부 수정한다면, 문제가 생기므로(예를 들면, month와 day에 맞지 않는 dayname이 할당 된다는가 하는)
    #  property에서 getter함수만 접근할 수 있게 하였다.
    @property
    def year(self):
        return self._year

    @property
    def month(self):
        return self._month

    @property
    def day(self):
        return self._day

    @property
    def hour(self):
        return self._hour

    @property
    def minute(self):
        return self._minute

    @property
    def dday(self):
        return self._dday

    # timechecker를 show 메쏘드와 cal_time_remaining 메쏘드에 decorate한다.
    # 객체의 정보를 보여줄 때(show 메쏘드) 일정까지 몇 시간 남았는지 알려줄 때(cal_time_remaining 메쏘드), 현재시간을 같이 출력할 것이다.
    def timechecker(func):
        def decorated(self):
            # 현재시간 출력
            cur_time = tm.strftime("%Y-%m-%d %H:%M", tm.localtime())
            print("현재시간은", cur_time, "입니다.")
            func(self)

        return decorated

    # 객체가 가지고 있는 정보들을 보여주는 메쏘드
    @timechecker
    def show(self):
        if self._mark is not None:
            print(self._mark, end='')
        if self._dday is None:
            print("{}년 {}월 {}일 {} {}시 {}분 \"{}\" 일정이 있습니다.\n"
                  .format(self._year, self._month, self._day, self._dayname, self._hour, self._minute, self._schedule))
        else:
            print("{}년 {}월 {}일 {} {}시 {}분 \"{}\" 일정이 있습니다.\n{}\n"
                  .format(self._year, self._month, self._day, self._dayname, self._hour, self._minute, self._schedule, self._dday))

    # 일정까지 남은 시간이 몇 시간인지 알려주는 메쏘드
    @timechecker
    def cal_time_remaining(self):
        delta = dt.datetime(self._year, self._month, self._day, self._hour, self._minute) - dt.datetime.now()
        totalseconds = delta.days * 24 * 3600 + delta.seconds
        a = tm.gmtime(totalseconds)
        print("%s 일정까지 남은 시간은 약 %s시간 %s분 입니다.\n" % (self._schedule, a.tm_hour, a.tm_min))

    # 오늘의 일정 목록을 뽑아 보여주는 메쏘드,
    # self 객체로의 접근이 아닌, 여러가지 객체들을 관리하는 것이어서, classmethod로 구현했다.
    @classmethod
    def show_today_schedule(cls):
        print("<금일 스케줄 목록>")
        check = 0
        today_schedules = []
        for schedule in Scheduler.schedules:
            today = dt.datetime.today()
            if (schedule.year, schedule.month, schedule.day) == (today.year, today.month, today.day):
                today_schedules.append(schedule)
                check = 1
        # 일정을 '시간'순서대로 보여주기 위한 정렬
        today_schedulues_sorted = sorted(today_schedules,
                                         key=lambda x: dt.datetime(x.year, x.month, x.day, x.hour, x.minute))

        for schedule in today_schedulues_sorted:
            print("%2s시 %2s분 %s" % (schedule.hour, schedule.minute, schedule.schedule))

        if check == 0:
            print("없습니다")
        
        print("----------------------------------------------------------\n")

    # 사용자가 입력한 날짜의 일정 목록을 뽑아 보여주는 메쏘드,
    # self 객체로의 접근이 아닌, 여러가지 객체들을 관리하는 것이어서, classmethod로 구현했다.
    @classmethod
    def show_someday_schedule(cls):
        print("일정을 조회할 날짜를 입력하세요")

        while 1:
            try:
                year = int(input("__년 : "))
                if year >= 0:
                    break
                else:
                    print("다시 입력해주세요")
            except:
                print("다시 입력해주세요")
                continue
        while 1:
            try:
                month = int(input("__월 : "))
                if 1 <= month <= 12:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue
        while 1:
            dic_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
            try:
                day = int(input("__일 : "))
                if 1 <= day <= dic_month[month]:
                    break
                else:
                    print("다시 입력해주세요")
            except ValueError:
                print("다시 입력해주세요")
                continue

        print("<%s년 %s월 %s일 스케줄 목록>" % (year, month, day))
        check = 0
        someday_schedules = []
        for schedule in Scheduler.schedules:
            if (schedule.year, schedule.month, schedule.day) == (year, month, day):
                someday_schedules.append(schedule)
                check = 1

        # 일정을 '시간'순서대로 보여주기 위한 정렬
        someday_schedulues_sorted = sorted(someday_schedules,
                                           key=lambda x: dt.datetime(x.year, x.month, x.day, x.hour, x.minute))

        for schedule in someday_schedulues_sorted:
            print("%2s시%2s분 %s" % (schedule.hour, schedule.minute, schedule.schedule))

        if check == 0:
            print("없습니다")
        
        print("----------------------------------------------------------\n")

    # 오늘로 부터 미래로 일주일 안의 일정 목록을 뽑아 보여주는 메쏘드,
    # self 객체로의 접근이 아닌, 여러가지 객체들을 관리하는 것이어서, classmethod로 구현했다.
    @classmethod
    def show_oneweek_schedule(cls):
        print("<일주일 간 일정들>")

        check = 0
        oneweek_schedules = []
        today = dt.datetime.today()
        # 시간/ 분을 0시 / 0분으로 맞추기 위한 작업. 이 작업을 해야만 당일에 있는 일정들을 보여주게 된다.
        today_tuned = dt.datetime(today.year, today.month, today.day)

        for schedule in Scheduler.schedules:
            schedule_day = dt.datetime(schedule.year, schedule.month, schedule.day)
            delta = schedule_day - today_tuned
            if 0 <= delta.days < 7:
                oneweek_schedules.append(schedule)
                check = 1

        # 일정을 '날짜'순서대로 보여주기 위한 정렬
        oneweek_schedules_sorted = sorted(oneweek_schedules, key= \
            lambda x: dt.datetime(x.year, x.month, x.day, x.hour, x.minute))

        for schedule in oneweek_schedules_sorted:
            print("%2s일 %s" % (schedule.day, schedule.schedule))

        if check == 0:
            print("없습니다")
            
        print("----------------------------------------------------------\n")

    # 일정을 삭제하는 소멸자.
    def __del__(self):
        print("=> ", end='')
        if self._mark is not None:
            print(self._mark, end='')
        print("{}년 {}월 {}일 {} {}시 {}분 \"{}\" 일정이 삭제되었습니다." \
              .format(self._year, self._month, self._day, self._dayname, self._hour, self._minute, self._schedule))
        print("----------------------------------------------------------\n")

Overwriting Scheduler.py


In [2]:
%run Scheduler.py

a = Scheduler.gen_schedule()   # 2020.5.18 20시30분 물리 과제
a.show()
a.schedule = "화학 과제"       # 물리과제가 아니라 화학과제였음을 깨닫고 바꾸는 상황 가정
a.show()

del Scheduler.schedules[a]     # reference counter가 문제가 되므로, schedules dictionary에서 제거 후 소멸자를 부른다.
del a

b = Scheduler.gen_schedule()    # 2020.5.17 23시59분 기초 빅데이터 프로그래밍 과제

c = Scheduler.gen_schedule()    # 2020.5.17 21시00분 과외
c.cal_time_remaining()

d = Scheduler.gen_schedule()    # 2020.5.20 23시59분 물리 과제
d.gen_dday()
print(d.dday, '\n')

e = Scheduler.gen_schedule()    # 2020.6.8 19시00분 (*중요*)엄마 생일 저녁식사
e.gen_mark()
e.gen_dday()
e.show()

f = Scheduler.gen_schedule()    # 2020.6.8 13시15분 (*중요*)미분기하학 퀴즈
f.gen_mark()
f.show()

Scheduler.show_today_schedule()     # 오늘날짜
Scheduler.show_someday_schedule()   # 2020.6.8
Scheduler.show_oneweek_schedule()   # 2020.5.16 ~ 2020.5.22

__년 : 2020
__월 : 5
__일 : 18
__시(0~23) : 20
__분 : 30
일정 내용: 물리 과제
=> 2020년 5월 18일 월요일 20시 30분 "물리 과제" 일정이 등록되었습니다.
----------------------------------------------------------

현재시간은 2020-05-17 19:46 입니다.
2020년 5월 18일 월요일 20시 30분 "물리 과제" 일정이 있습니다.

=> 2020년 5월 18일 월요일 20시 30분 "화학 과제" 일정으로 변경되었습니다.
----------------------------------------------------------

현재시간은 2020-05-17 19:46 입니다.
2020년 5월 18일 월요일 20시 30분 "화학 과제" 일정이 있습니다.

=> 2020년 5월 18일 월요일 20시 30분 "화학 과제" 일정이 삭제되었습니다.
----------------------------------------------------------

__년 : 2020
__월 : 5
__일 : 17
__시(0~23) : 23
__분 : 59
일정 내용: 기초빅데이터프로그래밍 과제
=> 2020년 5월 17일 일요일 23시 59분 "기초빅데이터프로그래밍 과제" 일정이 등록되었습니다.
----------------------------------------------------------

__년 : 2020
__월 : 5
__일 : 17
__시(0~23) : 21
__분 : 0
일정 내용: 과외
=> 2020년 5월 17일 일요일 21시 0분 "과외" 일정이 등록되었습니다.
----------------------------------------------------------

현재시간은 2020-05-17 19:46 입니다.
과외 일정까지 남은 시간은 약 1시간 13분 입니다.

__년 : 2020
__월 : 5
__일 : 20
__시(0~23) : 23
__분